In [ ]:
# Project team ID : PTID-CDS-OCT-24-2145
# Project: PRCP-1006-HomeLoanDef
# Project Members : Teja and Iqbal

**Importing Necessary Libraries**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,classification_report, roc_auc_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
import gc
import warnings
warnings.filterwarnings('ignore')



**Loading Datasets**

In [ ]:
application_train=pd.read_csv(r"C:\Users\Teja\Downloads\PRCP-1006-HomeLoanDef\Data\application_train.csv")
bureau=pd.read_csv(r"C:\Users\Teja\Downloads\PRCP-1006-HomeLoanDef\Data\bureau.csv")
bureau_balance=pd.read_csv(r"C:\Users\Teja\Downloads\PRCP-1006-HomeLoanDef\Data\bureau_balance.csv")
credit_card_balance=pd.read_csv(r"C:\Users\Teja\Downloads\PRCP-1006-HomeLoanDef\Data\credit_card_balance.csv")
installments_payments=pd.read_csv(r"C:\Users\Teja\Downloads\PRCP-1006-HomeLoanDef\Data\installments_payments.csv")
POS_CASH_balance=pd.read_csv(r"C:\Users\Teja\Downloads\PRCP-1006-HomeLoanDef\Data\POS_CASH_balance.csv")
previous_application=pd.read_csv(r"C:\Users\Teja\Downloads\PRCP-1006-HomeLoanDef\Data\previous_application.csv")

In [ ]:
application_train.head()

** Defining Function of insection of Initial data **




---



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def inspect_data(df, df_name="Dataset"):
    print(f"\n------{df_name} INITIAL ANALYSIS------\n")

    # Shape
    print(f"Shape of {df_name}: {df.shape}")

    # Data types and basic info
    print(f"\nData types and basic info for {df_name}:")
    df.info()

    # Missing values
    print(f"\nMissing values in {df_name}:")
    missing_values = df.isnull().sum()
    missing_values = missing_values[missing_values > 0]
    if len(missing_values) > 0:
        print("Columns with missing values:")
        print(missing_values)
    else:
        print(f"{df_name} has no missing values")

    # Descriptive analysis
    print(f"\nDescriptive analysis of {df_name}:")
    print(df.describe(include='all'))

    # Data distributions
    print(f"\nDistribution of numeric features in {df_name}:")
    numeric_columns = df.select_dtypes(include=['number']).columns
    if not numeric_columns.empty:
        print("Skewness of numeric features:")
        print(df[numeric_columns].skew())

        # Histograms for numeric features
        df[numeric_columns].hist(figsize=(15, 10), bins=30)
        plt.suptitle(f"Histograms for Numeric Features in {df_name}", fontsize=16)
        plt.show()

        # Boxplots for numeric features
        for col in numeric_columns:
            plt.figure(figsize=(8, 4))
            sns.boxplot(x=df[col])
            plt.title(f"Boxplot for {col} in {df_name}")
            plt.show()
    else:
        print(f"{df_name} has no numeric features to analyze for distribution.")

    # Value counts for categorical features
    print(f"\nValue counts for categorical features in {df_name}:")
    categorical_columns = df.select_dtypes(include=['object', 'category']).columns
    if not categorical_columns.empty:
        for col in categorical_columns:
            print(f"\nValue counts for {col}:")
            print(df[col].value_counts())

            # Bar plot for categorical features
            plt.figure(figsize=(10, 6))
            df[col].value_counts().plot(kind='bar', color='skyblue', edgecolor='black')
            plt.title(f"Bar Plot for {col} in {df_name}")
            plt.xlabel(col)
            plt.ylabel("Count")
            plt.show()
    else:
        print(f"{df_name} has no categorical features to analyze.")

    # Correlation analysis
    print(f"\nCorrelation analysis for numeric features in {df_name}:")
    if not numeric_columns.empty:
        correlation_matrix = df[numeric_columns].corr()
        print(correlation_matrix)

        # Heatmap for correlation matrix
        plt.figure(figsize=(12, 8))
        sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap="coolwarm", square=True)
        plt.title(f"Correlation Heatmap for {df_name}")
        plt.show()
    else:
        print(f"{df_name} has no numeric features to analyze for correlations.")

    # check for unique values per column
    print(f"\nUnique values per column in {df_name}:")
    print(df.nunique())


Analysing the data

In [ ]:
#application_train.head()
inspect_data(application_train," Application Data")

EDA of application_train
-------------------------
1) Rows = 307511
2) Columns= 122
3) dtypes: float64(65), int64(41), object(16)
4) Skewness of significant columns
	->TARGET=3.078159 -> highy positive skewed

In [ ]:
inspect_data(bureau," Bureau Data")

EDA of bureau
-------------------------
1) Rows = 1716428
2) Columns= 17
3) dtypes: float64(8), int64(6), object(3)
4) Skewness of significant columns
	SK_ID_CURR->0.001063 (Nearly symmentric)
	SK_ID_BUREAU-> -0.007498 (Nearly symmentric)

In [ ]:
inspect_data(bureau_balance," Bureau balance Data")


EDA of Bureau balance Data
-------------------------
1) Rows = 27299925
2) Columns= 3
3) dtypes: int64(2), object(1)
4) Skewness of significant columns
	SK_ID_CURR->-0.372188(Nearly symmentric)

In [ ]:
inspect_data(credit_card_balance," Credit card balance data")


In [ ]:
inspect_data(installments_payments," Installments payments data")


In [ ]:
inspect_data(POS_CASH_balance," POS cash balance data")


In [ ]:
inspect_data(previous_application," Previous application data")

In [ ]:
target_counts=application_train.TARGET.value_counts()/len(application_train)*100

In [ ]:
plt.figure(figsize=(8, 6))
plt.pie(target_counts,autopct='%1.1f%%',labels=["NOT DEFAULTER (0)", "DEFAULTER (1)"],colors=['blue','orange'],startangle=90,explode=[0, 0.1])
plt.title("Distribution of Target Variable")
plt.legend(title="Categories",alignment='right',loc='upper left')
plt.show()
                      # HERE THE DATA IS  IMBALANCED (91.9%(LOAN REPAYED-0)AND 8.1%(LOAN NOT REPAYED-1)AND WE NEED TO HANDLE THIS PROBLEM )

TAGET column in application_train Proportion
TARGET

0    282686(91.9%)

1     24825(8.1%)

->There is high imbalance of TARGET feature. Machine learning models
may become biased toward predicting the majority class (0),
leading to poor performance on the minority class.

->Oversample the 0 values by using SMOTE

In [ ]:
sns.histplot(application_train.AMT_INCOME_TOTAL,kde=True,bins=30)
plt.xlabel("AMT_INCOME_TOTAL")
plt.ylabel('count of applicants')
plt.show()

In [ ]:
# there is lot of skewness on the right side.
# most of the data point are between 0 and 0.2 million

In [ ]:
pd.set_option('display.max_columns',None)

In [ ]:
namecontract=application_train.NAME_CONTRACT_TYPE.value_counts()/len(application_train.NAME_CONTRACT_TYPE)*100

In [ ]:
plt.pie(namecontract,autopct='%1.1f%%',labels=["cash Loans", " Revolving loans"],colors=["blue", "orange"],startangle=90,explode=[0, 0.1])
plt.legend(title='categories',alignment='right',loc='upper left')
plt.title("Distribution of NAME_CONTRACT_TYPE")
plt.show()

In [ ]:
sns.histplot(application_train.AMT_CREDIT,color='orange',bins=30)
plt.title("Distribution of AMT_CREDIT")
plt.xlabel("AMT_CREDIT")
plt.ylabel("count of applicants")
plt.show()

In [ ]:
# The distribution is highly right skewed indicating outliers
# The majority of applicants have creadit between 0.0 to 1.25 million
# There is peak at 0.5 millioin which means most have this amount as most commoon applicants

In [ ]:
namesuit=application_train.NAME_TYPE_SUITE.value_counts()/len(application_train.NAME_TYPE_SUITE)*100

In [ ]:
plt.figure(figsize=(8, 6))
sns.barplot(namesuit,color='orange')
plt.title("Distribution of NAME_TYPE_SUITE")
plt.xlabel("NAME_TYPE_SUITE")
plt.ylabel("count of applicants")
plt.show()

In [ ]:
# The unaccompanied has majority of loan applications
# The family has the most number of applications after unaccompained
# like wise with the spouse and partner

In [ ]:
plt.figure(figsize=(12, 6))

sns.barplot(application_train.NAME_INCOME_TYPE.value_counts()/len(application_train.NAME_INCOME_TYPE)*100,color='red')
plt.title("Distribution of NAME_INCOME_TYPE")
plt.xlabel("NAME_INCOME_TYPE")
plt.ylabel("count of applicants")
plt.show()

In [ ]:
# The number of people who are applied to loan are working professionals followed by commercial associate adn pensioner

In [ ]:
plt.figure(figsize=(12,6))
sns.barplot(application_train.NAME_EDUCATION_TYPE.value_counts()/len(application_train.NAME_EDUCATION_TYPE)*100,color='purple')

In [ ]:
# The people with secondary education are tend to apply more for loans than
#  the higher education

In [ ]:
plt.figure(figsize=(12,6))
sns.barplot(application_train.NAME_FAMILY_STATUS.value_counts()/len(application_train.NAME_FAMILY_STATUS)*100,color='green')
plt.title("Distribution of NAME_FAMILY_STATUS")
plt.xlabel("NAME_FAMILY_STATUS")
plt.ylabel("count of applicants")
plt.show()

In [ ]:
# there are maority of the married people who applied for loan and follwed by singles and rest of all are minority

In [ ]:
plt.figure(figsize=(12,6))
sns.barplot(application_train.NAME_HOUSING_TYPE.value_counts()/len(application_train.NAME_HOUSING_TYPE)*100,color='blue')
plt.title("Distribution of NAME_HOUSING_TYPE")
plt.xlabel("NAME_HOUSING_TYPE")
plt.ylabel("count of applicants")
plt.show()

In [ ]:
# most of the applicants are living in house apratments ans restof all are minority amount of people livving with parents,municipal apartments etc

In [ ]:
sns.histplot(application_train.DAYS_BIRTH/-365,color='orange',bins=50)
plt.title("Distribution of DAYS_BIRTH")
plt.xlabel("DAYS_BIRTH")
plt.ylabel("count of applicants")

In [ ]:
# There are variety of age groups who are applied to loans however maximun of the applicants are in between in the age groups of mid 30's and 40's

In [ ]:
sns.histplot(application_train.DAYS_EMPLOYED/-365,color='orange',bins=50)

In [ ]:
# The data looks strange(we have -1000.66 years(-365243 days) of employment which is impossible) looks like there is data entry error

In [ ]:
# Identify and flag errors in DAYS_EMPLOYED
application_train['DAYS_EMPLOYED_ERROR'] = application_train['DAYS_EMPLOYED'] == 365243
application_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace=True)

# Calculate years of employment
application_train['DAYS_EMPLOYED'] = (application_train['DAYS_EMPLOYED']) /(-365)

# Create the histogram
sns.histplot(application_train.DAYS_EMPLOYED, bins=30)
plt.xlabel('DAYS_EMPLOYED')
plt.ylabel('Count of Applicants')
plt.show()

In [ ]:
# the people with less work experience less than 2 years

# DATA CLEANING FUNTIONS

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

def data_cleaning(df, df_name="Dataset"):
      print(f"--- Cleaning {df_name} ---")

      # 1. Drop columns with more than 50% missing values
      threshold = 0.5 * len(df)
      columns_to_drop = df.columns[df.isnull().sum() > threshold]
      df = df.drop(columns=columns_to_drop)
      print(f"Columns {list(columns_to_drop)} dropped due to >50% missing values.")

      # 2. Impute missing values
      for col in df.columns:
          if df[col].dtype in ['int64', 'float64']:  # Numerical column
              df[col].fillna(df[col].median(), inplace=True)
          else:  # Categorical column
              df[col].fillna(df[col].mode()[0], inplace=True)
      print(f"Imputed missing values in {df_name}.")

      # 3. Outlier detection and handling
      numerical_features = df.select_dtypes(include=['int64', 'float64']).columns
      for col in numerical_features:
          print(df[col].unique())
          df[col] = pd.to_numeric(df[col], errors='coerce')
          df[col].fillna(0, inplace=True)
          Q1 = df[col].quantile(0.25)
          Q3 = df[col].quantile(0.75)
          IQR = Q3 - Q1
          LowerBound = Q1 - 1.5 * IQR
          UpperBound = Q3 + 1.5 * IQR
          outliers = (df[col] < LowerBound) | (df[col] > UpperBound)
          if outliers.any():
              df[col] = np.where(outliers, np.nan, df[col])
              df[col].fillna(df[col].median(), inplace=True)
              print(f"Replaced outliers with median in column {col}.")
      print("end of outliers")

      print("Start of high correlation")
      # Keep only numeric columns
      numeric_df = df.select_dtypes(include=['number'])

      # Check if there are numeric columns to process
      if numeric_df.empty:
          print("No numeric columns found for correlation.")
      else:
          # Threshold for high correlation
          threshold = 0.8

          # Compute correlation matrix
          correlation_matrix = numeric_df.corr()

          # Find highly correlated pairs
          to_drop = set()
          for i in range(len(correlation_matrix.columns)):
              for j in range(i):
                  if abs(correlation_matrix.iloc[i, j]) > threshold:
                      colname = correlation_matrix.columns[i]
                      to_drop.add(colname)

          # Drop identified columns
          df.drop(columns=list(to_drop), inplace=True)
      print("end of high correlation")

      print("start of Standardize")
      # 6. Standardize numerical columns
      scaler = StandardScaler()
      numerical_features = df.select_dtypes(include=['number']).columns
      df[numerical_features] = scaler.fit_transform(df[numerical_features])
      print(f"Standardized numerical columns in {df_name}.")
      print("end of Standardize")

      print("start of categorical")
      # 5. Convert binary categorical columns to 'category' dtype
      binary_columns = df.select_dtypes(include=['object']).columns
      for col in binary_columns:
          if df[col].nunique() <= 2:
              df[col] = df[col].astype('category')
              print(f"Converted {col} to 'category' dtype.")
      print("end of categorical")

      # 7. One-hot encoding for remaining categorical features
      df = pd.get_dummies(df, drop_first=True)
      print(f"Applied one-hot encoding to categorical columns in {df_name}.")

      # 8. Rename columns for consistency
      df.columns = [col.replace(" ", "_").lower() for col in df.columns]
      print(f"Renamed columns in {df_name} for consistency.")

      print("start of Remove duplicates")
      # 4. Remove duplicates
      original_shape = df.shape  # Corrected
      df.drop_duplicates(inplace=True)
      if original_shape != df.shape:
          print(f"Removed {original_shape[0] - df.shape[0]} duplicate rows.")
      print("end of Remove duplicates")

      return df


In [ ]:
target=application_train['TARGET']
application_train=application_train.drop(columns=['TARGET'])
application_train_cleaned=data_cleaning(application_train,df_name="Application Data") # SEPERATING THE TARGET FILE FOR THE
application_train_cleaned['TARGET']=target


**Clean Data**

In [ ]:
application_train_cleaned.describe()


In [ ]:
bureau_cleaned=data_cleaning(bureau,df_name="Bureau Data")


In [ ]:
bureau_cleaned.describe()

In [ ]:
bureau_balance_cleaned=data_cleaning(bureau_balance,df_name="Bureau balance Data")


In [ ]:
credit_card_balance_cleaned=data_cleaning(credit_card_balance,df_name="Credit card balance")

In [ ]:
installments_payments_cleaned=data_cleaning(installments_payments,df_name="Installments payments data")


In [ ]:
POS_CASH_balance_cleaned=data_cleaning(POS_CASH_balance,df_name="POS cash balance data")


In [ ]:
previous_application_cleaned=data_cleaning(previous_application,df_name="Previous application data")


**Finding relationship columns**

In [ ]:
print("application_train_cleaned\n", application_train_cleaned.columns)
print("bureau_cleaned.columns\n",bureau_cleaned.columns)
print("bureau_balance_cleaned\n",bureau_balance_cleaned.columns)
print("POS_CASH_balance_cleaned\n",POS_CASH_balance_cleaned.columns)
print("credit_card_balance_cleaned\n",credit_card_balance_cleaned.columns)
print("previous_application_cleaned\n",previous_application_cleaned.columns)
print("installments_payments_cleaned\n",installments_payments_cleaned.columns)

In [ ]:
print(f"Unique sk_id_curr in application_train_cleaned: {application_train_cleaned['sk_id_curr'].nunique()}")
print(f"Shape of application_train_cleaned: {application_train_cleaned.shape}")
print(f"Unique sk_id_curr in bureau_cleaned: {bureau_cleaned['sk_id_curr'].nunique()}")
print(f"Unique sk_id_bureau in bureau_cleaned: {bureau_balance_cleaned['sk_id_bureau'].nunique()}")
print(f"Unique sk_id_bureau in bureau_balance_cleaned: {bureau_balance_cleaned['sk_id_bureau'].nunique()}")
print(f"Unique sk_id_prev in previous_application_cleaned: {previous_application_cleaned['sk_id_prev'].nunique()}")
print(f"Unique sk_id_prev in installments_payments_cleaned: {installments_payments_cleaned['sk_id_prev'].nunique()}")


# **MERGE DATASETS on (sk_id_curr, sk_id_bureau, sk_id_prev)**

In [ ]:
application_train_cleaned.TARGET.value_counts()


In [ ]:
data_1 = application_train_cleaned.merge(bureau_cleaned, on="sk_id_curr", how="left")
print(f"After merging bureau_cleaned: {data_1.shape}")
bureau_cleaned=None
application_train_cleaned=None

In [ ]:
data_2 = data_1.merge(bureau_balance_cleaned, on="sk_id_bureau", how="left")
print(f"After merging bureau_balance_cleaned: {data_2.shape}")
bureau_balance_cleaned=None
data_1=None


In [ ]:
data_3 = data_2.merge(credit_card_balance_cleaned, on="sk_id_curr", how="left")
print(f"After merging credit_card_balance_cleaned: {data_3.shape}")
credit_card_balance_cleaned=None
data_2=None

In [ ]:
data_4 = data_3.merge(previous_application_cleaned, on=["sk_id_curr", "sk_id_prev"], how="left")
print(f"After merging previous_application_cleaned: {data_4.shape}")
previous_application_cleaned=None
data_3=None


In [ ]:
data_5 = data_4.merge(installments_payments_cleaned, on=["sk_id_curr", "sk_id_prev"], how="left")
print(f"After merging installments_payments_cleaned: {data_5.shape}")
installments_payments_cleaned=None
data_4=None

In [ ]:
data_5.describe()

In [ ]:
data=data_5

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
print(data.shape)

In [ ]:
print(data.columns)

In [ ]:
data.describe()

In [ ]:
data.TARGET.value_counts()

In [ ]:
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.feature_selection import VarianceThreshold

# Removing the TARGET column
features = data.drop(columns=['TARGET'])

# Removing low-variance features
variance_filter = VarianceThreshold(threshold=0.01)
high_variance_features = variance_filter.fit_transform(features.select_dtypes(include=['float64', 'int64']))
selected_columns = features.columns[variance_filter.get_support(indices=True)]
filtered_data = features.loc[:, selected_columns]

# Calculate VIF
vif_data = pd.DataFrame()
vif_data["Feature"] = filtered_data.columns
vif_data["VIF"] = [
    variance_inflation_factor(filtered_data.values, i) for i in range(filtered_data.shape[1])
]


In [ ]:
variance_filter=None
high_variance_features=None
selected_columns=None
vif_data=None
features=None
data=None

In [ ]:
target.value_counts()

In [ ]:
 #Train-Test Split
#X = data.drop(columns=['TARGET', 'SK_ID_CURR'])
#y = data['TARGET']
X = filtered_data.drop(columns=['sk_id_curr'])
y=target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

In [ ]:
# Define Model Evaluation Function
from sklearn.metrics import classification_report

# Updated Model Evaluation Function
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name="Model"):

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    gc.collect()
    print(f"\n{model_name} Classification Report:")
    print(classification_report(y_test, y_pred, target_names=["Not Default", "Default"]))

    # Confusion Matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
    plt.title(f"{model_name} Confusion Matrix")
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.show()

    return model

In [ ]:
#Hyperparameter Tuning Function
from sklearn.model_selection import RandomizedSearchCV

def hyperparameter_tuning(model, param_distributions, X_train, y_train, n_iter=50, cv=5, scoring='f1'):
    """
    Perform hyperparameter tuning using RandomizedSearchCV.

    Args:
        model: The machine learning model to be tuned.
        param_distributions: Dictionary with parameters to sample during tuning.
        X_train: Training features.
        y_train: Training target.
        n_iter: Number of parameter settings to sample (default is 50).
        cv: Number of cross-validation folds (default is 5).
        scoring: Scoring metric for evaluation (default is 'f1').

    Returns:
        best_model: The best estimator after hyperparameter tuning.
    """

    random_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_distributions,
        n_iter=n_iter,
        cv=cv,
        scoring=scoring,
        n_jobs=-1,
        verbose=1,
        random_state=42  # For reproducibility
    )

    random_search.fit(X_train, y_train)
    print("Best Hyperparameters:", random_search.best_params_)
    best_model = random_search.best_estimator_
    return best_model


# DECISION TREE

In [ ]:
 #1. Decision Tree
 dt_param_grid = {
     #'max_depth': [5, 10, 15, 20, None],
     'max_depth': [5],
     'min_samples_split': [2, 5, 10],
     'min_samples_leaf': [1, 2, 4],
     'criterion': ['gini', 'entropy']
 }

 dt_model = DecisionTreeClassifier(random_state=42)
 best_dt_model = hyperparameter_tuning(dt_model, dt_param_grid, X_train, y_train)
 evaluate_model(best_dt_model, X_train, X_test, y_train, y_test, model_name="Tuned Decision Tree")


# LOGISTIC REGRESSION

In [ ]:
from sklearn.linear_model import LogisticRegression
# 2. Logistic Regression
# Define the parameter grid for Logistic Regression
lr_param_grid = {
    'penalty': ['l1', 'l2', 'elasticnet', None],  # Regularization type
    'C': [0.01, 0.1, 1, 10, 100],               # Regularization strength
    'solver': ['liblinear', 'saga'],            # Solvers that support l1 and elasticnet
    'max_iter': [100, 200, 500]                 # Number of iterations
}

# Instantiate the Logistic Regression model
lr_model = LogisticRegression(random_state=42)

# Perform hyperparameter tuning using the defined function
best_lr_model = hyperparameter_tuning(lr_model, lr_param_grid, X_train, y_train, n_iter=20, scoring='f1')

# Evaluate the tuned Logistic Regression model
evaluate_model(best_lr_model, X_train, X_test, y_train, y_test, model_name="Tuned Logistic Regression")


# XGBOOSTING

In [ ]:

# 5. XGBoost
xgb_param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
best_xgb_model = hyperparameter_tuning(xgb_model, xgb_param_grid, X_train, y_train)
evaluate_model(best_xgb_model, X_train, X_test, y_train, y_test, model_name="Tuned XGBoost")

# LIGHT GRADIENT BOOSITNG

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report

from lightgbm import LGBMClassifier
Lgbmc=LGBMClassifier()
Lgbmc.fit( X_train,y_train)
ypred=Lgbmc.predict(X_test)
acccuracy=accuracy_score(ypred,y_test)
acccuracy   # LGBM BOOSTING IS GIVING GOOD ACCURACY

# RANDOM FOREST CLASSIFIER

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RFC = RandomForestClassifier()
RFC.fit(X_train,y_train)
y_pred=RFC.predict(X_test)
accuracy=accuracy_score(y_pred,y_test)
accuracy

# SUPPORT VECTOR CLASSIFIER

In [ ]:
from sklearn.svm import SVC
model = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42,max_iter=50)  # Using RBF kernel
model.fit(X_train, y_train)

sy_pred=model.predict(X_test)
accuracy=accuracy_score(sy_pred,y_test)
accuracy

# Project Submission: Machine Learning Model Evaluation for HomeLoanDef Dataset

**Objective:** To evaluate and identify the best-performing machine learning model for predicting loan defaults using the provided HomeLoanDef dataset.

**Models Evaluated:**
1.   Support Vector Classifier (SVC)
2.   Random Forest Classifier (RFC)
3.   Light Gradient Boosting Machine (LGBM)
4.   XGBoost Classifier - Accracy
5.   Logistic Regression (LR)
6.   Decision Tree Classifier (DTC) - Accuracy

**Analysis:**
*   Among all models, ***Light Gradient Boosting Machine (LGBM) and XGBoost Classifier achieved the highest accuracy***
*   Both models demonstrate superior performance, outperforming others in handling the complexity of the dataset.
*   Logistic Regression and Decision Tree models had the lowest accuracies, indicating they are less suited for this problem.
*   Random Forest and Support Vector Classifier models performed competitively but fell slightly short of the top performers.

**Conclusion:**

Based on the accuracy metric:

*   Light Gradient Boosting Machine (LGBM) and XGBoost Classifier are equally effective for the HomeLoanDef dataset.
*   Both models are recommended for deployment, considering their high accuracy and efficiency.





